In [1]:
## Importing Libraries
import numpy as np
import pandas as pd
import os
import random
from operator import itemgetter
import copy
import time
import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Steps for training

* Initialize the tranformer
* Create the dataloaders for training, test & validation
* Intialize models
* Intialize the training
* Evaluate the model on test set
* Desiging the KPI : [ Confusion Matrix ]

In [2]:
# Initializing the transformer
transformer = {
    'original': transform.Compose([
                                 transform.Resize((220, 220)),
                                 transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                 transform.RandomRotation(5),
                                 transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8,0.8)),
                                 transform.ToTensor(),
                                 transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178), 
                                               (0.3268945515155792, 0.29282665252685547, 0.29053378105163574)),
]),
    'test' : transform.Compose([
                                 transform.Resize((220, 220)),
                                 transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                 transform.ToTensor(),
                                 transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178), 
                                               (0.3268945515155792, 0.29282665252685547, 0.29053378105163574)),
])}

####################################################################################################
bs = 50
train_path = '../input/flower-dataset/train'
original = ImageFolder(train_path, transform=transformer['original'])
test_path = '../input/flower-dataset/test'
test = ImageFolder(test_path, transform=transformer['test'])
# Splitting the train & test dataset
train, val = train_test_split(original, test_size=0.2, shuffle=True, random_state=43)
loaders = {
    'train': DataLoader(train, batch_size=bs, num_workers=0, pin_memory=True),
    'val': DataLoader(val, batch_size=bs, num_workers=0, pin_memory=True),
    'test': DataLoader(test, batch_size=bs, num_workers=0, pin_memory=True)
}

dataset_sizes = {
    'train': len(train),
    'val': len(val), 
    'test': len(test),
}

In [3]:
# accuracy to measure the model
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1) 
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)), preds

#save the losses for further visualization
losses = {'train':[], 'val':[]}
accuracies = {'train':[], 'val':[]}
lr = []

In [4]:
def train(epochs, model):    
  print('----------------------Creating a model -----------------------')
  PATH = '../input/ssl-model/resnet18_best.pth'
  model.to(device)  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay = 1e-5)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
  since = time.time()
  best_model = copy.deepcopy(model.state_dict())
  state_dict = torch.load(PATH,map_location=device)
  best_model = model.load_state_dict(state_dict, strict=False)
  best_acc = 0.0
  for epoch in range(epochs):
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()      
      running_loss = 0.0
      running_corrects = 0.0
      for inputs, labels in loaders[phase]:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase=='train'):
          outp = model(inputs)
          _, pred = torch.max(outp, 1)
          loss = criterion(outp, labels) 
          if phase == 'train':
            loss.backward()
            optimizer.step()
        running_loss += loss.item()*inputs.size(0)
        running_corrects += torch.sum(pred == labels.data)

      if phase == 'train':
          acc = 100. * running_corrects.double() / dataset_sizes[phase]
          scheduler.step(acc)

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double()/dataset_sizes[phase]
      losses[phase].append(epoch_loss)
      accuracies[phase].append(epoch_acc)
      if phase == 'train':
        print('Epoch: {}/{}'.format(epoch+1, epochs))
      print('{} - loss:{}, accuracy{}'.format(phase, epoch_loss, epoch_acc))
      lr.append(scheduler._last_lr)
        
      if phase == 'val':
        print('Time: {}m {}s'.format((time.time()- since)//60, (time.time()- since)%60))
        print('=='*31)
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model = copy.deepcopy(model.state_dict())
    #scheduler.step() 
      state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
          }
      Path = './saved_model.pth'
      torch.save(state, Path)
  time_elapsed = time.time() - since
  print('CLASSIFIER TRAINING TIME {}m {}s'.format(time_elapsed//60, time_elapsed%60))
  print('=='*31)
  

**GoogleNet**

In [5]:
googlenet = torchvision.models.googlenet(pretrained=True)
for param in googlenet.parameters():
  param.grad_requires = False

googlenet.fc = nn.Linear(in_features=googlenet.fc.in_features, out_features=len(original.classes), bias=True)

### Training 

In [6]:
epochs = 50
models = googlenet
model = train(epochs=epochs, model=models)

### Loss and Accuracy Plots

In [7]:
#  Plot train lacc and test accuracy 
plt.figure(figsize=(12,8))
plt.plot(accuracies['train'],label = 'train Acc')
plt.plot(accuracies['val'],label  = 'test Acc')
plt.legend()

In [8]:
#  Plot train and test loss
plt.figure(figsize=(12,8))
plt.plot(losses['train'],label = 'train loss')
plt.plot(losses['val'],label  = 'test loss')
plt.legend()

### Evaluation on test data

In [8]:
model = googlenet

In [9]:
def validation_step(batch):
        images,labels = batch
        images,labels = images.to(device),labels.to(device)
        out = model(images)                                      
        loss = F.cross_entropy(out, labels)                    
        acc,preds = accuracy(out, labels)                       
        
        return {'val_loss': loss.detach(), 'val_acc':acc.detach(), 
                'preds':preds.detach(), 'labels':labels.detach()}
    
def test_prediction(outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()           
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()             
        # combine predictions
        batch_preds = [pred for x in outputs for pred in x['preds'].tolist()] 
        # combine labels
        batch_labels = [lab for x in outputs for lab in x['labels'].tolist()]  
        
        return {'test_loss': epoch_loss.item(), 'test_acc': epoch_acc.item(),
                'test_preds': batch_preds, 'test_labels': batch_labels}

In [10]:
@torch.no_grad()
def test_predict(model, test_loader):
    model.eval()
    # perform testing for each batch
    outputs = [validation_step(batch) for batch in test_loader] 
    results = test_prediction(outputs)                          
    print('test_loss: {:.4f}, test_acc: {:.4f}'
          .format(results['test_loss'], results['test_acc']))
    
    return results['test_preds'], results['test_labels']

In [11]:
model.to(device)
preds,labels = test_predict(model, loaders['test'])

### KPI Metrics

In [20]:
import json

with open('../input/cat-json/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

class_names = original.classes
# changing categories to their actual names 
for i in range(0,len(class_names)):
    class_names[i] = cat_to_name.get(class_names[i])

In [12]:
report = classification_report(labels, preds,
                               output_dict=True,
                               target_names=original.classes)
report_df = pd.DataFrame(report).transpose()


In [13]:
pd.set_option("display.max_rows", None)
report_df.head(134)

In [14]:
# Plot confusion matrix
cm  = confusion_matrix(labels, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(19,19),cmap=plt.cm.Blues)
plt.xticks(range(len(original.classes)), original.classes, fontsize=16)
plt.yticks(range(len(original.classes)), original.classes, fontsize=16)
plt.xlabel('Predicted Label',fontsize=18)
plt.ylabel('True Label',fontsize=18)
plt.show()